<a href="https://colab.research.google.com/github/Cami050/proyecto_exploracion_datos/blob/main/Proyecto_de_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FileNotFoundError: [Errno 2] No such file or directory: '/content/DELITOS_INFORMÁTICOS_20251209.csv'